In [ ]:
from urllib.parse import urlparse
from services.knowledge_base.web_scrape import map_url, scrape_url
from services.chat.chat import llm_response
from typing import List
from humanloop import Humanloop
from dotenv import load_dotenv
import os

""" URL TO MARKDOWN """

load_dotenv()

async def url_markdown(urls: List[str]):
    global scraped_content
    for url in urls:
        print("url:", url)
        # Get domain name for agent name
        domain = urlparse(url).netloc
        agent_name = domain.replace("www.", "")
        print("agent_name:", agent_name)
        
        # Map and scrape the URL
        mapped_url = await map_url(url)
        print("\n\nmapped_url:", mapped_url)
        scraped_content = await scrape_url(mapped_url[:100], user_id="user_2mmXezcGmjZCf88gT2v2waCBsXv")
        web_content = ""
        for i in scraped_content:
            web_content += i['content']
        
        # Save content to markdown file
        filename = f"{agent_name}.md"
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(web_content)
        
        print(f"Content saved to {filename}")
    
    return filename

# Example usage:
urls = [
    "https://www.perlonlabs.com/"
]

response = await url_markdown(urls)

print("response", response)

In [ ]:
import uuid
import requests
from urllib.parse import urlparse
from services.knowledge_base.web_scrape import map_url, scrape_url
from services.chat.chat import llm_response
from typing import List, Dict
import openai
from humanloop import Humanloop
from dotenv import load_dotenv
import os

load_dotenv()

client = Humanloop(
    api_key=os.getenv("HUMANLOOP_API_KEY"),
)

async def create_opening_line(scraped_content: str) -> str:
    # Assuming you have OpenAI API setup
    user_prompt = f"""
    # Instructions:
    Based on this website content, create an engaging opening line for a customer service agent, like  <example_opening_line>"Welcome to WeCreate👋! Looking to boost your brand's online presence? What's your biggest goal right now?</example_opening_line>
    IMPORTANT: You must only respond with the opening line, no other text. No speech marks either, only the text.
    # Website content:
    {scraped_content[:1000]}"""
    
    system_prompt = "You are an expert at creating opening lines for customer service agents. You are given a website content and you are tasked with creating a friendly opening line for a customer service agent."
    
    response = await llm_response(system_prompt, user_prompt)
    return response

async def create_agent_instructions(scraped_content: str) -> str:
    user_prompt = f"""
    # Instructions:
    Based on this website content, create a short sentence that begins with the name of the Company, and a short description saying what it does, like "WeCreate, a branding & digital marketing agency based Singapore, Hong Kong, and the UK."
    IMPORTANT: You must only respond with the instructions, no other text.
    # Website content:
    {scraped_content[:1000]}
    """
    
    system_prompt = "You are an expert at creating instructions for customer service agents. You are given a website content and you are tasked with creating instructions for a customer service agent."
    
    response = await llm_response(system_prompt, user_prompt)
    return response

async def create_agents_from_urls(urls: List[str]):
    global scraped_content
    responses = []
    for url in urls:
        print("url:",url)
        # Get domain name for agent name
        domain = urlparse(url).netloc
        agent_name = domain.replace("www.", "")
        print("agent_name:",agent_name)
        # Map and scrape the URL
        mapped_url = await map_url(url)
        print("\n\nmapped_url:",mapped_url)
        scraped_content = await scrape_url(mapped_url[:100], user_id="user_2mmXezcGmjZCf88gT2v2waCBsXv")
        web_content = ""
        for i in scraped_content:
            web_content += i['content']
        web_content = web_content[:10000]

        # Generate opening line from content
        opening_line = await create_opening_line(web_content)
        print("\n\nopening line:",opening_line)

        company_details = await create_agent_instructions(web_content)

        print("\n\ncompany details:",company_details)
        ## abstracted away lead form content, as that's determined by the user and stored in the metadata

        humanloop_prompt = client.prompts.get(
            id="pr_BiPy5GsclDI8uAzmHR8S7",
        )
        agent_instructions = humanloop_prompt.template[0]['content'].replace("{{company_details}}", company_details)

        # Create data source dict
        data_source = [{
            "id": str(uuid.uuid4()),
            "title": url,
            "data_type": "web"
        }]
        
        features = {
                    "lead_gen": {
                        "enabled": True
                    },
                    "callTransfer": {
                        "enabled": False
                    },
                    "notifyOnInterest": {
                        "enabled": False
                    },
                    "appointmentBooking": {
                        "enabled": False
                    }
                }
        
        form_fields = {
                "fields": [
                    {
                    "type": "text",
                    "label": "Full name"
                    },
                    {
                    "type": "email",
                    "label": "Email address"
                    }
                ]
            }
        
        """ POST to create new agent """
        # Prepare payload
        payload = {
            "agentName": agent_name,
            "agentPurpose": "text-chatbot-agent",
            "dataSource": data_source,
            "openingLine": opening_line,
            "language": "en-GB", 
            "voice": "Alex K",
            "voiceProvider": "cartesia",
            "instructions": agent_instructions,
            "features": features,
            "form_fields": form_fields,
            "company_name": agent_name
        }

        # POST request
        response = requests.post(
            "https://flowon.ai/api/v1/livekit/new_agent",
            json=payload,
            headers={"x-user-id": "user_2mmXezcGmjZCf88gT2v2waCBsXv"}
        )
        
        if response.status_code == 200:
            print(f"Successfully created agent for {url}")
            responses.append(response.json())  # Add response to list
        else:
            print(f"Failed to create agent for {url}: {response.text}")

# Example usage:
urls = [
    # "https://www.quickfiredigital.com/",
    # "https://www.footprintdigital.co.uk/",
    "https://cosmarketingagency.com/",
    "https://www.timdhodges.ca/",
    "https://www.liftconversions.com/",
    "https://ezedim.com/"

]

response = await create_agents_from_urls(urls)
# agent_id = response['data'][0]['id']

# print("agent_id", agent_id)

In [ ]:
import os
import re

def duplicate_files(folder_path, original_name, new_name, variable_name, new_value):
    """
    Duplicates HTML and JS files while modifying a specified variable.
    
    Args:
        folder_path (str): Path to the folder containing the files
        original_name (str): Base name of the original files (without extension)
        new_name (str): Base name for the new files (without extension)
        variable_name (str): Name of the variable to modify
        new_value: New value to set for the variable
    """
    
    # Handle both HTML and JS files
    for ext in ['.html', '.js']:
        original_file = os.path.join(folder_path, f"{original_name}{ext}")
        new_file = os.path.join(folder_path, f"{new_name}{ext}")
        
        # Check if original file exists
        if not os.path.exists(original_file):
            print(f"Original file {original_file} not found")
            continue
            
        try:
            # Read the original file
            with open(original_file, 'r', encoding='utf-8') as file:
                content = file.read()
            
            # Replace the variable value
            # This pattern matches variable declarations in both JS and HTML
            pattern = f'(const|let|var)?\s*{variable_name}\s*=\s*["\']?([^"\';]+)["\']?'
            new_content = re.sub(pattern, f'\g<1> {variable_name} = "{new_value}"', content)
            
            # Write to new file
            with open(new_file, 'w', encoding='utf-8') as file:
                file.write(new_content)
                
            print(f"Successfully created {new_file}")
            
        except Exception as e:
            print(f"Error processing {original_file}: {str(e)}")


# Example usage
for url in urls:
    duplicate_files('./my_folder', 'original', 'copy', 'apiKey', 'new-api-key-value')

In [ ]:
""" CHAT WITH AGENT"""
from services.chat.lk_chat import lk_chat_process

async def test_chat(message: str , agent_id: str):
    room_name = "test_room_45iii22ij6ii"

    try:
        # Get the response generator
        response_generator = lk_chat_process(message, agent_id, room_name)
        
        print("\n\nAgent response:\n\n")
        # Stream the responses
        async for chunk in response_generator:
            # In a real application, you might want to send this to a websocket or other stream
            print('\033[96m' + chunk + '\033[0m', end='', flush=True)   

    except Exception as e:
        print(f"Error occurred: {str(e)}")

await test_chat(
    message="payment message standards to be observed by all PSPs", 
    agent_id="a706a185-fe32-4331-b35a-d241d29e9154")

In [ ]:
from services.chat.chat import similarity_search

response = await similarity_search(
    query="tell me about the company", 
    data_source={'web': ['https://nogood.io'], 'text_files': []},
    user_id="user_2mmXezcGmjZCf88gT2v2waCBsXv")

response

### Recall Meeting bots

In [ ]:
import requests

# API endpoint
url = "https://us-west-2.recall.ai/api/v1/bot/"

# Your authorization token
token = "209a894215526cd61d70ed2f77df6cb39f6b0e4d"

# Request headers
headers = {
    "Authorization": f"Token {token}",
    "Content-Type": "application/json"
}

# Request payload
payload = {
    "meeting_url": "https://meet.google.com/stn-gobs-xcr",
    "bot_name": "Bot"
}

# Send POST request
response = requests.post(url, headers=headers, json=payload)

# Check response
print(response.status_code)
print(response.json())

In [ ]:
bot_joining = response.json()

bot_joining

In [ ]:
import requests

url = "https://us-west-2.recall.ai/api/v1/bot/e09e55e2-a067-473d-80be-18b9725ba7b8/"

headers = {
    "accept": "application/json",
    "Authorization": "209a894215526cd61d70ed2f77df6cb39f6b0e4d"
}

response = requests.get(url, headers=headers)

print(response.text)

In [ ]:
response.json()